In [ ]:
!pip install google-cloud-aiplatform google-cloud-storage
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout, Flatten, Attention, Input
from tensorflow.keras.layers import TimeDistributed
import pandas as pd
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import root_mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
from datetime import datetime
import gc
import psutil
from tensorflow import keras

In [ ]:
from google.cloud import storage

BUCKET_NAME = "gold_predictions_web"  # เปลี่ยนเป็นชื่อ Bucket ของคุณ

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
print(f"เชื่อมต่อกับ GCS Bucket: {BUCKET_NAME}")

เชื่อมต่อกับ GCS Bucket: gold_predictions_web


In [ ]:
import requests
nowDate=datetime.now()
startTime=f'{nowDate.year-10}-{nowDate.month}-{nowDate.day}'
endTime=f'{nowDate.year}-{nowDate.month}-{nowDate.day}'
startTime_1=f'{nowDate.day}-{nowDate.month}-{nowDate.year-10}'
endTime_1=f'{nowDate.day}-{nowDate.month}-{nowDate.year}'
url_currency = f"https://gold-predictions.duckdns.org/currency/get/?start={startTime}&end={endTime}&cache=False"
url_Gold_usd = f"https://gold-predictions.duckdns.org/finnomenaGold/get-gold-data/?db_choice=1&start={startTime_1}&end={endTime_1}&cache=False"
url_Gold_thb = f"https://gold-predictions.duckdns.org/finnomenaGold/get-gold-data/?db_choice=0&start={startTime_1}&end={endTime_1}&cache=False"
# เรียก API และดึงข้อมูล
response = requests.get(url_currency)

# ตรวจสอบว่าการเรียก API สำเร็จหรือไม่
if response.status_code == 200:
    data_currency = response.json()  # แปลงข้อมูลเป็น JSON
    data_list = data_currency["data"]
    df_currency = pd.DataFrame(data_list)  # สร้าง DataFrame
    print(df_currency.tail())  # แสดงตัวอย่างข้อมูล
else:
    print(f"Error: {response.status_code}")

response = requests.get(url_Gold_usd)

# ตรวจสอบว่าการเรียก API สำเร็จหรือไม่
if response.status_code == 200:
    data_Gold_usd = response.json()  # แปลงข้อมูลเป็น JSON
    data_list = data_Gold_usd["data"]
    df_Gold_usd = pd.DataFrame(data_list)  # สร้าง DataFrame
    print(df_Gold_usd.tail())  # แสดงตัวอย่างข้อมูล
else:
    print(f"Error: {response.status_code}")

response = requests.get(url_Gold_thb)

# ตรวจสอบว่าการเรียก API สำเร็จหรือไม่
if response.status_code == 200:
    data_Gold_thb = response.json()  # แปลงข้อมูลเป็น JSON
    data_list = data_Gold_thb["data"]
    df_Gold_thb = pd.DataFrame(data_list)  # สร้าง DataFrame
    print(df_Gold_thb.tail())  # แสดงตัวอย่างข้อมูล
else:
    print(f"Error: {response.status_code}")

df_currency=df_currency[['date','price']]
df_currency.columns = ['Date', 'price_exchange']

df_Gold_thb=df_Gold_thb[['timestamp','price']]
df_Gold_thb = df_Gold_thb.copy()
df_Gold_thb['timestamp'] = df_Gold_thb['timestamp'].apply(lambda x: x // 1000 if x > 10**12 else x)
df_Gold_thb['timestamp'] = pd.to_datetime(df_Gold_thb['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok')
df_Gold_thb['timestamp'] = df_Gold_thb['timestamp'].dt.strftime('%d/%m/%Y')
df_Gold_thb.columns = ['Date', 'Thai_Gold']

df_Gold_usd=df_Gold_usd[['timestamp','close_price']]
df_Gold_usd = df_Gold_usd.copy()
df_Gold_usd['timestamp'] = df_Gold_usd['timestamp'].apply(lambda x: x // 1000 if x > 10**12 else x)
df_Gold_usd['timestamp'] = pd.to_datetime(df_Gold_usd['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok')
df_Gold_usd['timestamp'] = df_Gold_usd['timestamp'].dt.strftime('%d/%m/%Y')
df_Gold_usd.columns = ['Date', 'Xau']


# รวม DataFrame ทั้งสามโดยใช้ date เป็นคีย์อ้างอิง
df_currency['Date'] = pd.to_datetime(df_currency['Date'], format='%Y-%m-%d').dt.normalize()
df_Gold_thb['Date'] = pd.to_datetime(df_Gold_thb['Date'], format='%d/%m/%Y').dt.normalize()
df_Gold_usd['Date'] = pd.to_datetime(df_Gold_usd['Date'], format='%d/%m/%Y').dt.normalize()
merged_df = (
    df_Gold_thb
    .merge(df_Gold_usd, on='Date', how='outer')
    .merge(df_currency, on='Date', how='outer')
)
merged_df = (
    merged_df
    .sort_values('Date')
    .groupby('Date', as_index=False)
    .first()  # เอาแถวแรกของแต่ละวัน
)
print(df_currency.dtypes)
print(df_Gold_thb.dtypes)
print(df_Gold_usd.dtypes)

merged_df[['Xau']] = merged_df[['Xau']].shift(1)
merged_df['Xau'] = merged_df['Xau'].ffill()
merged_df.dropna(inplace=True)
merged_df.to_csv('acvs.csv')
dupes = merged_df[merged_df.duplicated('Date', keep=False)]
print(f"จำนวนวันที่ซ้ำ: {len(dupes)}")
print(dupes.sort_values('Date'))

          id        date  price   open    high     low  percent  diff
6884  194967  2025-04-17  33.34  33.10  33.445  33.055   0.8164  0.27
6885  194968  2025-04-18  33.35  33.35  33.460  32.950   0.0300  0.01
6886  194970  2025-04-19  33.39  33.35  33.450  33.320   0.1100  0.04
6887  194971  2025-04-20  33.38  33.39  33.440  33.310  -0.0400 -0.01
6888  194972  2025-04-21  33.07  33.35  33.350  33.050  -0.9300 -0.31
         id      timestamp    price  close_price  high_price  low_price  \
3041  90318  1744502400000  3238.07      3223.90     3238.07    3210.60   
3042  90319  1744588800000  3221.80      3212.34     3245.23    3194.59   
3043  90320  1744675200000  3212.71      3250.51     3250.80    3210.20   
3044  90323  1744761600000  3252.86      3349.52     3357.01    3250.34   
3045  90325  1744848000000  3349.87      3314.93     3355.33    3276.44   

      volume  volume_weight_avg  num_transactions      date  \
3041   262.0          3219.8965               262  13-04-25   
304

In [ ]:
movingavg=[0]
lag=[3]
seq=[1]
month_data=[True]
# Load the data
for month_onoff in range(len(month_data)):

    data = merged_df.copy()

# Convert 'Date' to datetime format
    data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')

    # Remove commas from the 'Xau' column and convert to float
    # data['Xau'] = data['Xau'].str.replace(',', '').astype(float)

    # Set 'Date' as the index
    data.set_index('Date', inplace=True)

    # Resample to monthly data and calculate open, max, and min for Thai_Gold
    monthly_data = data.resample('ME').agg({
        'Thai_Gold': ['first', 'max', 'min'],
        'price_exchange': 'mean',
        'Xau': 'mean'
    })

    # Rename columns for clarity
    monthly_data.columns = [
        'Thai_Gold_Open', 'Thai_Gold_Max', 'Thai_Gold_Min',
        'Price_Exchange_Mean', 'Xau_Mean'
    ]

    # Add the correct 'Thai_Gold_Open' for the first day of each month
    first_of_month = data.resample('MS').first()['Thai_Gold']
    monthly_data['Thai_Gold_Open'] = first_of_month.values
    monthly_data.to_csv('Month.csv')
    # Add a 'Month' column with numeric month values
    if month_data[month_onoff] == True:
      monthly_data['Month'] = monthly_data.index.strftime('%m')

    # Calculate moving averages for 3, 5, and 7 months
    for moving_num in range(len(movingavg)):
      if movingavg[moving_num] >=1:
        monthly_data['MA_3'] = monthly_data['Thai_Gold_Open'].rolling(window=3).mean()
      if movingavg[moving_num] >=2:
        monthly_data['MA_5'] = monthly_data['Thai_Gold_Open'].rolling(window=5).mean()
      if movingavg[moving_num] >=3:
        monthly_data['MA_7'] = monthly_data['Thai_Gold_Open'].rolling(window=7).mean()
      # Rearrange columns

      # Display the final DataFrame
      monthly_data.dropna(inplace=True)
      print(monthly_data)
      # Feature=['Month','Price_Exchange_Mean', 'Xau_Mean', 'MA_3', 'MA_5', 'MA_7']
      Target=['Thai_Gold_Open', 'Thai_Gold_Max', 'Thai_Gold_Min']
      data_clone=monthly_data.copy()
      for lagFea in range(len(lag)):
        X = monthly_data.drop(columns=Target)
        y = monthly_data[Target].values
        for i in range(1, lag[lagFea]+1):  # สร้างย้อนหลัง 7 วัน
            X[f'Thai_Gold_Open_Lag_{i}'] = data_clone['Thai_Gold_Open'].shift(i)
            X[f'Thai_Gold_Max_Lag_{i}'] = data_clone['Thai_Gold_Max'].shift(i)
            X[f'Thai_Gold_Min_Lag_{i}'] = data_clone['Thai_Gold_Min'].shift(i)
        ASD=y
        X = X.dropna().reset_index(drop=True)
        y = np.array(y, dtype=np.float64)

        # ลบ 3 index แรก
        y = y[lag[lagFea]:]
        scaler_X = MinMaxScaler(feature_range=(0,1))
        scaler_y = MinMaxScaler(feature_range=(0,1))
        X = scaler_X.fit_transform(X)
        y = scaler_y.fit_transform(y)
        print(len(y))
        for lookbackFea in range(len(seq)):
          def create_sequences(X, lookback):
              X_seq = []
              for i in range(len(X) - lookback):
                  X_seq.append(X[i:i+lookback])
                  # print(i+lookback)
              return np.array(X_seq)
          lookback = seq[lookbackFea]  # ดูย้อนหลัง 30 วัน
          n_future = 1   # พยากรณ์ 7 วันข้างหน้า   # พยากรณ์ 7 วันข้างหน้า
          X_seq = create_sequences(X, lookback)

          # กำหนดขนาดข้อมูล
          test_size = 15     # ขนาด Test Set
          val_size = 20      # ขนาด Validation Set
          train_size = X_seq.shape[0] - (val_size + test_size)  # ขนาด Training Set

          # แบ่งข้อมูลเป็น Training, Validation และ Test ตามลำดับเวลา
          X_train= X_seq[:train_size]
          X_val = X_seq[train_size:train_size + val_size]
          X_test = X_seq[train_size + val_size:]

          # ตรวจสอบขนาดของชุดข้อมูล
          # print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
          # print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
          # print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
          MODEL_NAME ='Model_month.keras'
          DESTINATION_DIR = "models"        # โฟลเดอร์ที่ต้องการเก็บโมเดลในเครื่อง
          os.makedirs(DESTINATION_DIR, exist_ok=True)
          destination_path = os.path.join(DESTINATION_DIR, MODEL_NAME)
          blob = bucket.blob(f'models_month/{MODEL_NAME}')
          blob.download_to_filename(destination_path)
          # โหลดกลับมาใช้งาน
          model = keras.models.load_model(destination_path)

            # พยากรณ์
          predictions = model.predict(X_test)

            # แปลงผลลัพธ์กลับสู่สเกลเดิม
          predictions_original = scaler_y.inverse_transform(predictions)



            Thai_Gold_Open  Thai_Gold_Max  Thai_Gold_Min  Price_Exchange_Mean  \
Date                                                                            
2015-04-30         18300.0        18700.0        18150.0            32.631663   
2015-05-31         18400.0        19300.0        18400.0            33.519400   
2015-06-30         18850.0        19050.0        18700.0            33.718846   
2015-07-31         18700.0        18700.0        17900.0            34.333132   
2015-08-31         18050.0        19450.0        18000.0            35.413977   
...                    ...            ...            ...                  ...   
2024-12-31         42900.0        43300.0        42150.0            34.112823   
2025-01-31         42550.0        44350.0        42550.0            34.203211   
2025-02-28         44500.0        46900.0        44500.0            33.753608   
2025-03-31         46300.0        49950.0        46250.0            33.722308   
2025-04-30         50500.0  

In [ ]:
df_last_15 = monthly_data.tail(15)

In [ ]:
df_last_15.reset_index(inplace=True)

In [ ]:
df_last_15 = df_last_15['Date'].reset_index(drop=True)

In [ ]:
predict = pd.DataFrame(predictions_original,columns=['open','high','low'])

In [ ]:
df_last_15=pd.concat([df_last_15, predict], axis=1)

In [ ]:
df_last_15.dtypes

,0
Date,datetime64[ns]
open,float32
high,float32
low,float32


In [ ]:
from datetime import datetime
import time
import pytz

# กำหนดโซนเวลา Indochina (ICT, UTC+7)
indochina_tz = pytz.timezone('Asia/Bangkok')

# เวลาปัจจุบันใน UTC
current_utc_time = datetime.utcnow().replace(tzinfo=pytz.utc)

# เวลาปัจจุบันใน Indochina Time (ICT, UTC+7)
current_ict_time = current_utc_time.astimezone(indochina_tz)

# แปลงเป็น Unix Timestamp (วินาที)
current_unix_time = int(time.time())

# ปรับฟอร์แมตของ Timestamp ICT ให้เป็น `YYYY-MM-DD HH:MM:SS.SSS +0700`
timestamp_ict_formatted = current_ict_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] + " +0700"

# เพิ่มคอลัมน์ลงใน df_last_350
df_last_15 = df_last_15.assign(
    timestamp=current_unix_time,  # Unix Timestamp (UTC)
    created_at=timestamp_ict_formatted  # Timestamp ICT แบบฟอร์แมตที่ต้องการ
)
df_last_15.rename(columns={"Date": "date"}, inplace=True)
# df_last_15["date"] = pd.to_datetime(df_last_15["date"])  # แปลงเป็น datetime64
# df_last_15["timestamp"] = df_last_15["date"].astype("int64") // 10**9  # แปลงเป็น Unix timestamp (วินาที)
df_last_15['month_predict'] = (df_last_15['date'] + pd.DateOffset(months=1)).dt.strftime('%Y-%m')
df_last_15["date"] = df_last_15["date"].astype(str).str[:16]  # ตัดให้ไม่เกิน 16 ตัวอักษร
df_last_15["month_predict"] = df_last_15["month_predict"].astype(str).str[:16]  # ตัดให้ไม่เกิน 16 ตัวอักษร
# แสดงผลลัพธ์
df_last_15=df_last_15.iloc[[-1]]
# df_last_15[['open', 'high', 'low']] = df_last_15[['open', 'high', 'low']].astype(float)

In [ ]:
print(df_last_15)

          date          open          high           low   timestamp  \
0   2024-02-29  34543.136719  36734.585938  33567.371094  1709164800   
1   2024-03-31  35024.566406  37273.632812  34022.820312  1711843200   
2   2024-04-30  35606.144531  37943.449219  34580.597656  1714435200   
3   2024-05-31  36880.761719  39407.617188  35818.421875  1717113600   
4   2024-06-30  39871.332031  42878.187500  38661.914062  1719705600   
5   2024-07-31  43176.320312  46746.281250  41833.917969  1722384000   
6   2024-08-31  45774.722656  49828.851562  44334.824219  1725062400   
7   2024-09-30  46465.710938  50667.250000  44999.578125  1727654400   
8   2024-10-31  46698.460938  50946.664062  45215.675781  1730332800   
9   2024-11-30  46984.593750  51292.636719  45500.886719  1732924800   
10  2024-12-31  47511.753906  51909.714844  46023.175781  1735603200   
11  2025-01-31  49070.621094  53767.378906  47530.253906  1738281600   
12  2025-02-28  49837.949219  54802.570312  48335.796875  174070

In [ ]:
from sqlalchemy import create_engine
# แปลง date เป็น datetime
df=monthly_data
df.reset_index(inplace=True)
df = df['Date'].reset_index(drop=True)
df = df.to_frame()
df= pd.to_datetime(df["Date"])
df = df.to_frame()
# ดึงวันที่ล่าสุดจาก DataFrame
last_date = df['Date'].max().date()  # เอาเฉพาะวันที่ (ไม่เอาเวลา)

# ดึงวันที่ปัจจุบัน
today = datetime.now().date()
if last_date == today:
  # 🔹 กำหนดค่าการเชื่อมต่อ PostgreSQL
  DB_USER = "postgres"  # ชื่อผู้ใช้ PostgreSQL
  DB_PASSWORD = "postgres"  # รหัสผ่าน
  DB_HOST = "34.133.61.28"  # ปกติใช้ "localhost" หรือ IP ของเซิร์ฟเวอร์
  DB_PORT = "5432"  # พอร์ตเริ่มต้นของ PostgreSQL
  DB_NAME = "postgres"  # ชื่อฐานข้อมูล
  TABLE_NAME = "predicts_month"  # ชื่อตารางที่ต้องการ INSERT

    # 🔹 สร้าง Connection ไปยัง PostgreSQL ด้วย SQLAlchemy
  engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
    # df_last_350['created_at'] = pd.to_datetime(df_last_350['created_at']).dt.tz_convert('Asia/Bangkok')

    # 🔹 Insert DataFrame ลงใน PostgreSQL (ถ้าตารางไม่มีให้สร้างอัตโนมัติ)
  df_last_15.to_sql(TABLE_NAME, engine, if_exists="append", index=False)

  print(f"✅ Insert DataFrame ลงใน PostgreSQL ตาราง {TABLE_NAME} สำเร็จ!")


✅ Insert DataFrame ลงใน PostgreSQL ตาราง predicts_month สำเร็จ!
